In [1]:
### Library Imports

import pandas as pd
import numpy as np
import math 
import seaborn as sns 
import matplotlib.pyplot as plt
from datetime import datetime
import yfinance as yf 
import matplotlib.dates as mdates

In [2]:
### Load in Stock Data

tickers = 'GOOG'
start_date = '2019-02-01'

def import_stock_data(tickers, start_date):
    data = pd.DataFrame()
    if len([tickers]) == 1:
        data[tickers] = yf.download(tickers, start_date)['Adj Close']
        data = pd.DataFrame(data)
    else:
        for t in tickers:
            data[t] = yf.download(tickers, start_date)['Adj Close']
    # Reset index to make headings in the same row
    data.reset_index(inplace = True)

    data['Date'] = pd.to_datetime(data['Date'])

    return data

stock_data = import_stock_data(tickers, start_date)
stock_data

[*********************100%%**********************]  1 of 1 completed


,Date,GOOG
0,2019-02-01,55.537498
1,2019-02-04,56.639999
2,2019-02-05,57.299500
3,2019-02-06,55.761501
4,2019-02-07,54.935501
...,...,...
1265,2024-02-12,148.729996
1266,2024-02-13,146.369995
1267,2024-02-14,147.139999
1268,2024-02-15,143.940002


In [3]:
### Load in Stock Data

tickers = 'GOOG'
start_date = '2019-02-01'

def import_stock_data(tickers, start_date):
    data = pd.DataFrame()
    if len([tickers]) == 1:
        data[tickers] = yf.download(tickers, start_date)['Adj Close']
        data = pd.DataFrame(data)
    else:
        for t in tickers:
            data[t] = yf.download(tickers, start_date)['Adj Close']
    # Reset index to make headings in the same row
    data.reset_index(inplace = True)

    data['Date'] = pd.to_datetime(data['Date'])

    return data

stock_data = import_stock_data(tickers, start_date)
stock_data

[*********************100%%**********************]  1 of 1 completed


,Date,GOOG
0,2019-02-01,55.537498
1,2019-02-04,56.639999
2,2019-02-05,57.299500
3,2019-02-06,55.761501
4,2019-02-07,54.935501
...,...,...
1269,2024-02-16,141.759995
1270,2024-02-20,142.199997
1271,2024-02-21,143.839996
1272,2024-02-22,145.320007


In [ ]:
### Function to display SMA and EMA

def MovingAverageCrossStrategy(stock_data, short_window, long_window, moving_avg_type, display_table = True):
    '''
    The function takes the stock symbol, time-duration of analysis, 
    look-back periods and the moving-average type(SMA or EMA) as input 
    and returns the respective MA Crossover chart along with the buy/sell signals for the given period.
    '''
    # stock_data - df containing dates and closing prices
    # short_window - (int)lookback period for short-term moving average. Eg: 5, 10, 20 
    # long_window - (int)lookback period for long-term moving average. Eg: 50, 100, 200
    # moving_avg_type - (str)the type of moving average to use ('SMA' or 'EMA')
    # display_table - (bool)whether to display the date and price table at buy/sell positions(True/False)

    # column names for long and short moving average columns
    short_window_col = str(short_window) + '_' + moving_avg_type
    long_window_col = str(long_window) + '_' + moving_avg_type

    # Loop to compute rolling mean cols
    if moving_avg_type == 'SMA':


